In [1]:
import gzip
import opener
import pandas as pd
import io
import requests
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from collections import Counter

In [2]:
url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2018/full.csv.gz"
req = requests.get(url)

In [3]:
t = req.content

In [4]:
s = gzip.decompress(t)

In [73]:
df = pd.read_csv(io.BytesIO(s))

C:\Users\dargo\AppData\Local\Temp\ipykernel_11260\798997524.py:1: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.BytesIO(s))


In [6]:
df

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2018-1,2018-01-03,1,Vente,109000.0,13.0,NaN,RUE GEN LOGEROT,1660,1000.0,...,Appartement,73.0,4.0,NaN,NaN,NaN,NaN,NaN,5.220440,46.200062
1,2018-1,2018-01-03,1,Vente,109000.0,13.0,NaN,RUE GEN LOGEROT,1660,1000.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,5.220440,46.200062
2,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,163.0,4.0,S,sols,NaN,NaN,949.0,5.408041,46.255562
3,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,163.0,4.0,AG,terrains d'agrément,JARD,Jardin d'agrément,420.0,5.408041,46.255562
4,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,51.0,2.0,AG,terrains d'agrément,JARD,Jardin d'agrément,420.0,5.408041,46.255562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335287,2018-1434017,2018-12-28,1,Vente,1192307.0,1.0,NaN,PAS CHOISEUL,2012,75002.0,...,Appartement,150.0,4.0,NaN,NaN,NaN,NaN,NaN,2.335139,48.867253
3335288,2018-1434018,2018-12-03,1,Vente,383000.0,12.0,NaN,RUE POISSONNIERE,7561,75002.0,...,Appartement,34.0,1.0,NaN,NaN,NaN,NaN,NaN,2.347989,48.869018
3335289,2018-1434019,2018-12-28,1,Vente,746000.0,18.0,NaN,RUE STE CROIX BRETONNERIE,8548,75004.0,...,Local industriel. commercial ou assimilé,60.0,0.0,NaN,NaN,NaN,NaN,NaN,2.356566,48.858267
3335290,2018-1434020,2018-12-13,1,Adjudication,645000.0,9.0,NaN,BD MORLAND,6559,75004.0,...,Appartement,54.0,2.0,NaN,NaN,NaN,NaN,NaN,2.364563,48.848333


In [80]:
# suppression valeur fonciere nulle

df.dropna(subset=['valeur_fonciere'], inplace=True)
df.dropna(subset=['longitude', 'latitude'], inplace=True)

In [101]:
# Selection des colonnes utiles

dataset = df[['id_mutation',
              'date_mutation',
              'nature_mutation',
              'adresse_nom_voie',
              'code_postal',
              'code_commune',
              'code_departement',
              'nombre_lots',
              'code_type_local',
              'surface_reelle_bati',
              'nombre_pieces_principales',
              'code_nature_culture',
              'code_nature_culture_speciale',
              'surface_terrain',
              'valeur_fonciere',
              'longitude',
              'latitude'
              ]]

In [82]:
# del df

# Cleaning

In [83]:
dataset.shape

(3201638, 17)

### Date mutation cleaning

In [102]:
# Passage de la colonnes date mutation en datetime puis en seconde pour avoir des int

dataset.loc[:, 'date_mutation'] = pd.to_datetime(dataset['date_mutation'], format='mixed')
dataset.loc[:, 'date_mutation'] = dataset['date_mutation'].apply(lambda x: x.timestamp())

In [103]:
# Remplissage des valeurs NaN nature_mutation et labelisation

dataset.loc[:, 'nature_mutation'] = dataset.nature_mutation.fillna("")

NM = preprocessing.LabelEncoder()
NM.fit(dataset['nature_mutation'])
dataset.loc[:, 'nature_mutation'] = NM.transform(dataset['nature_mutation'])

### Adresse nom voie cleaning

In [104]:
abrv_voie = pd.read_csv(filepath_or_buffer='Data_Files/ABREVIATION_VOIE.csv', sep=',').values

In [105]:
dataset.loc[:, 'adresse_nom_voie'] = dataset.adresse_nom_voie.fillna("")

codex_voie = list()
for i in dataset.adresse_nom_voie.values:

    list_nom_rue = i.split(' ')
    premier_valeur_liste = list_nom_rue[0]

    if premier_valeur_liste in abrv_voie:
        codex_voie.append(premier_valeur_liste)
    else:
        codex_voie.append('AUTRE')

dataset.loc[:, 'prefixe_voie'] = codex_voie

C:\Users\dargo\AppData\Local\Temp\ipykernel_11260\90543416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,'prefixe_voie'] = codex_voie


CV = preprocessing.LabelEncoder()
CV.fit(dataset.prefixe_voie)
dataset.loc[:,'prefixe_voie'] = CV.transform(dataset.prefixe_voie)

### Multiple easy Cleaning

In [106]:
dataset.loc[:, 'code_postal'] = dataset.code_postal.fillna(0)

In [107]:
dataset.loc[:, 'surface_reelle_bati'] = dataset.surface_reelle_bati.fillna(0)
dataset.loc[:, 'surface_terrain'] = dataset.surface_terrain.fillna(0)
dataset.loc[:, 'code_type_local'] = dataset.code_type_local.fillna(0)
dataset.loc[:, 'nombre_pieces_principales'] = dataset.nombre_pieces_principales.fillna(0)

In [108]:
dataset.loc[:, 'code_nature_culture'] = dataset.code_nature_culture.fillna("")
dataset.loc[:, 'code_nature_culture_speciale'] = dataset.code_nature_culture_speciale.fillna("")

### Latitude & longitude

In [109]:
dataset.longitude.isna().sum()

0

In [110]:
dataset.latitude.isna().sum()

0

In [170]:
dataset.dtypes

id_mutation                      object
date_mutation                    object
nature_mutation                  object
adresse_nom_voie                 object
code_postal                     float64
code_commune                     object
code_departement                 object
nombre_lots                       int64
code_type_local                 float64
surface_reelle_bati             float64
nombre_pieces_principales       float64
code_nature_culture              object
code_nature_culture_speciale     object
surface_terrain                 float64
valeur_fonciere                 float64
longitude                       float64
latitude                        float64
prefixe_voie                     object
dtype: object

# Construction data set modele

In [171]:
classe_liste_nature_mutation = list(dict.fromkeys(list(dataset.nature_mutation.values)))

In [172]:
classe_liste_prefixe_voie = list(dict.fromkeys(codex_voie))

In [173]:
classe_liste_code_culture = pd.read_csv(filepath_or_buffer='Data_Files/CODE_CULTURE.csv', sep=',')
classe_liste_code_culture = list(classe_liste_code_culture['Code_nature_culture'].to_dict().values())

In [174]:
classe_liste_code_culture_spe = pd.read_csv(filepath_or_buffer='Data_Files/CODE_CULTURE_SPECIALE.csv', sep=',')
classe_liste_code_culture_spe = list(classe_liste_code_culture_spe['CODE_CULTURE_SPECIALE'].to_dict().values())

In [175]:
Nom_colonnes_from_dataset = [
    'date_mutation',
    'code_postal',
    'code_commune',
    'code_departement',
    'nombre_lots',
    'code_type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'surface_terrain',
    'valeur_fonciere',
    'longitude',
    'latitude'
]

In [176]:
colonnes = Nom_colonnes_from_dataset + classe_liste_nature_mutation + classe_liste_prefixe_voie + classe_liste_code_culture + classe_liste_code_culture_spe

In [50]:
test_id = dataset.id_mutation[0]

In [51]:
test_id

'2018-1'

In [165]:
TEST = dataset.loc[dataset['id_mutation'] == test_id]

In [187]:
TEST

,id_mutation,date_mutation,nature_mutation,adresse_nom_voie,code_postal,code_commune,code_departement,nombre_lots,code_type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,code_nature_culture_speciale,surface_terrain,valeur_fonciere,longitude,latitude,prefixe_voie
0,2018-1,1514937600.0,Vente,RUE GEN LOGEROT,1000.0,1053,1,2,2.0,73.0,4.0,,,0.0,109000.0,5.22044,46.200062,RUE
1,2018-1,1514937600.0,Vente,RUE GEN LOGEROT,1000.0,1053,1,1,3.0,0.0,0.0,,,0.0,109000.0,5.22044,46.200062,RUE


In [181]:
assert 1 == len(pd.unique(TEST.date_mutation))
assert 1 == len(pd.unique(TEST.valeur_fonciere))
assert 1 == len(pd.unique(TEST.code_postal))
assert 1 == len(pd.unique(TEST.code_commune))
assert 1 == len(pd.unique(TEST.code_departement))

In [203]:
# valeur prefixe voie

tmp_abbrev_voie = TEST.loc[:, 'prefixe_voie']
ajout_abbrev_voie = []

for abbrev_voie in classe_liste_prefixe_voie:
    ajout_abbrev_voie.append(TEST.loc[TEST['prefixe_voie'] == abbrev_voie, 'prefixe_voie'].size)

In [196]:
TEST.loc[TEST['prefixe_voie'] == 'ALL', 'prefixe_voie'].size

0

In [197]:
a = []

In [202]:
a = a + [3]
a

[2, 2, 3, 3]

In [177]:
tmp = pd.DataFrame(columns=colonnes)

In [185]:
tmp

,date_mutation,code_postal,code_commune,code_departement,nombre_lots,code_type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,valeur_fonciere,...,TOURB,TOUYA,VADC,VAGUE,VANIL,VAOC,VCHAS,VDQS,VIGNE,VIVIE


In [184]:
tmp.valeur_fonciere = TEST.loc[0, 'valeur_fonciere']

In [19]:
# Sortie de la colonne resultat

## Y = df.valeur_fonciere